> BoTorch makes no particular assumptions on what kind of model is being used, so long as is able to produce *samples from a posterior over outputs given an input x*.

> BoTorch abstracts away from the particular form of the posterior by providing a simple Posterior API that **only requires** implementing an **rsample()** method for sampling from the posterior.

In [11]:
import torch

from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior


In [2]:
class SimpleCustomGP(ExactGP, GPyTorchModel):
    _num_outputs = 1
    
    def __init__(self, train_X, train_Y):
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood()) # <- MRO!
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=train_X.shape[-1]))
        self.to(train_X) # Device
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

.forward() is the method called during the (PyTorch) training loop to get the output, or the posterior. 

## Python: Method Resolutio Order (MRO)
determines the order of inherited methods (in a multiple inheritance case)  
Note that `__mro__()` method is only visible for a class, not for a class instance.

In [13]:
SimpleCustomGP.__mro__

(__main__.SimpleCustomGP,
 gpytorch.models.exact_gp.ExactGP,
 gpytorch.models.gp.GP,
 gpytorch.module.Module,
 botorch.models.gpytorch.GPyTorchModel,
 botorch.models.model.Model,
 torch.nn.modules.module.Module,
 abc.ABC,
 object)

<details>
  <summary>ExactGP.__init__()</summary>

According to the `__mro__()`, `super().__int__()` is calling the `__init__()` of ExactGP, which takes **train_inputs, train_targets, and likelihood** as input.

```>>> ExactGP.__init__??```
```
>>>
Signature: ExactGP.__init__(self, train_inputs, train_targets, likelihood)
Docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(self, train_inputs, train_targets, likelihood):
        if train_inputs is not None and torch.is_tensor(train_inputs):
            train_inputs = (train_inputs,)
        if train_inputs is not None and not all(torch.is_tensor(train_input) for train_input in train_inputs):
            raise RuntimeError("Train inputs must be a tensor, or a list/tuple of tensors")
        if not isinstance(likelihood, _GaussianLikelihoodBase):
            raise RuntimeError("ExactGP can only handle Gaussian likelihoods")

        super(ExactGP, self).__init__()
        if train_inputs is not None:
            self.train_inputs = tuple(tri.unsqueeze(-1) if tri.ndimension() == 1 else tri for tri in train_inputs)
            self.train_targets = train_targets
        else:
            self.train_inputs = None
            self.train_targets = None
        self.likelihood = likelihood

        self.prediction_strategy = None
File:      ~/anaconda3/envs/tm38/lib/python3.8/site-packages/gpytorch/models/exact_gp.py
Type:      function

```
</details>

In [18]:
from botorch.fit import fit_gpytorch_model

def _get_and_fit_simple_custom_gp(Xs, Ys, **kwargs):
    model = SimpleCustomGP(Xs[0], Ys[0])
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    return model

For a custom (non-GP) model, you need to build your own fitting loop.  
(https://botorch.org/tutorials/fit_model_with_torch_optimizer)
